In [2]:
import pandas as pd
import numpy as np
import requests
import io

Inicialmente, extraímos os dados. A tabela de saneamento básico será extraída via requisição por API e as demais estão em arquivos csv

In [3]:
def getSanitationData():

    url = "https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/UNICEF,WASH_HOUSEHOLDS,1.0/.WS_PPL_H-B+WS_PPL_S-ALB+WS_PPL_S-SM+WS_PPL_W-ALB+WS_PPL_W-SM+WS_PPS_S-OD..._T?format=csv&labels=both"
    data = requests.get(url).content
    csv = pd.read_csv(io.StringIO(data.decode("utf-8")))
    csv.to_csv(r"../data/external/sanitation_data")

    return csv

Ao instanciar as tabelas, já tratamos algumas inconsistências. Como padronizar os nomes de algumas colunas, retirar colunas não necessárias para o nosso dataset e tratar de dados nulos e uniformizar a nálise temporal de 1990 à 2016.

In [4]:
sanitation_data = getSanitationData()

def handleCountryNameIDH(country):
    return country[1:]

def handleCountryNameCountry(country):
    return country[:-1]

country_data = pd.read_csv('../data/external/countries of the world.csv')
country_data = country_data[['Country','Region','Climate']]
country_data["Country"] = country_data["Country"].map(handleCountryNameCountry)
country_data.to_csv(r'../data/interim/country_data.csv')

IDH_data = pd.read_csv("../data/external/Human Development Index.csv")
IDH_data.drop( ["2017"], axis = 1, inplace = True )
IDH_data["Country"] = IDH_data["Country"].map(handleCountryNameIDH)
IDH_data.rename({'Country':'country', 'HDI Rank':'HDIRank'}, axis=1, inplace=True)
IDH_data.to_csv(r'../data/interim/IDH_data.csv')

sanitation_data.rename({'Geographic area': 'country', 'TIME_PERIOD':'year', 'Service Type':'ServiceType', 'Unit of measure':'UnitOfMeasure'}, axis=1, inplace=True)
sanitation_data = sanitation_data[['REF_AREA','country','INDICATOR','Indicator','ServiceType','year', 'UnitOfMeasure', 'OBS_VALUE']]
sanitation_data.drop(sanitation_data[sanitation_data.year > 2016].index, inplace=True)
sanitation_data.to_csv(r"../data/interim/sanitation_data.csv")

mortality_data = pd.read_excel('../data/external/global_mortality.xlsx')

mortality_data.rename(columns=lambda x: x.replace(' (%)', ''), inplace=True)
mortality_data.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)


Agora, vamos filtrar os dados de modo que a coluna país de cada tabela (saneamento, mortalidade e IDH) seja uma chave estrangeira para a tabela central de países.

In [5]:
for country in IDH_data["country"]:
    
    if (country not in country_data["Country"].to_numpy()):

        filtered_data = IDH_data["country"] != country
        IDH_data = IDH_data[filtered_data]
        
for country in sanitation_data["country"].unique():
    
    if (country not in country_data["Country"].to_numpy()):
    
        filtered_data = sanitation_data["country"] != country
        sanitation_data = sanitation_data[filtered_data]

for country in mortality_data["country"].unique():

    if (country not in country_data["Country"].to_numpy()):

        filtered_data = mortality_data["country"] != country
        mortality_data = mortality_data[filtered_data]

#criar uma coluna year e transformar as informações de IDH para esse modelo
newIDH_data = pd.melt(IDH_data.reset_index(), id_vars=['HDIRank', 'country'], var_name='year', value_name='HDI_VALUE')
IDH_data = newIDH_data[newIDH_data.year.str.contains("index") == False]

IDH_data.dropna(subset = ["HDI_VALUE"], inplace=True)

C:\Users\guilh\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Por fim, vamos exportar os arquivos alterados para a pasta processed

In [6]:
sanitation_data.to_csv(r'../data/processed/sanitation_data.csv')

IDH_data.to_csv(r'../data/processed/IDH_data.csv')

mortality_data.to_csv(r'../data/processed/mortality_data.csv')

country_data.to_csv(r'../data/processed/countries_data.csv')